# Exploration de la source de données

In [8]:
import pandas as pd
from datetime import date, timedelta
import os

In [9]:
#Racine des fichiers quotidiens
BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'

#Dates de disponibilité des fichiers
START_DATE = date(2020, 1, 22) #start date
END_DATE = date(2020, 3, 12) #End date

#Répertoire de sauvegarde des fichiers bruts
RAWFILES_DIR = '../data/raw/'
PROCESSED_DIR = '../data/processed/'

## Boucle de récupération des fichiers

In [10]:
delta = END_DATE - START_DATE 

for i in range(delta.days + 1):
    day = START_DATE + timedelta(days=i)
    day_label = day.strftime("%m-%d-%Y")
    virus_df = pd.read_csv(BASE_URL.format(day_label), sep=',', parse_dates=['Last Update'])
    virus_df.to_csv(os.path.join(RAWFILES_DIR, day_label + '.csv'), index=False)
    
    #print(day_label)

In [11]:
virus_df.dtypes

Province/State            object
Country/Region            object
Last Update       datetime64[ns]
Confirmed                  int64
Deaths                     int64
Recovered                  int64
Latitude                 float64
Longitude                float64
dtype: object

## Constitution de la table de références lat/long

In [12]:
import glob

df_list = []

#Lecture des fichiers récupérés et sélection de ceux qui ont une lat/long
for file in glob.glob(os.path.join(RAWFILES_DIR, '*.csv')):
    virus_df = pd.read_csv(file, sep=',')
    if 'Latitude' in virus_df.columns and 'Longitude' in virus_df.columns:
        df_list.append(virus_df)

all_df = pd.concat(df_list)

(all_df[['Province/State', 'Country/Region', 'Latitude', 'Longitude']]
 .drop_duplicates(subset=['Province/State', 'Country/Region'])
 .sort_values(by=['Country/Region', 'Province/State'])
 .to_csv(os.path.join(PROCESSED_DIR, 'lat_long_table.csv'), index=False))

## Construction d'une table unique

In [13]:
df_list = []
latlong_df=pd.read_csv(os.path.join(PROCESSED_DIR, "lat_long_table.csv"))

for file in glob.glob(os.path.join(RAWFILES_DIR, "*.csv")):
    virus_df = pd.read_csv(file, sep=",")
    if not("Latitude" in virus_df.columns and "Longitude" in virus_df.columns): #ça nous interesse et on veut les garder
        virus_df = virus_df.merge(latlong_df,on=['Province/State', 'Country/Region'], how='left')

    df_list.append(virus_df.assign(source=os.path.basename(file)))
    
all_df=pd.concat(df_list)

all_df.to_csv(os.path.join(PROCESSED_DIR, 'all_data.csv'), index=False)

In [14]:
all_df


,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,source
0,Anhui,Mainland China,2020-01-22 17:00:00,1.0,NaN,NaN,31.8257,117.2264,01-22-2020.csv
1,Beijing,Mainland China,2020-01-22 17:00:00,14.0,NaN,NaN,40.1824,116.4142,01-22-2020.csv
2,Chongqing,Mainland China,2020-01-22 17:00:00,6.0,NaN,NaN,30.0572,107.8740,01-22-2020.csv
3,Fujian,Mainland China,2020-01-22 17:00:00,1.0,NaN,NaN,26.0789,117.9874,01-22-2020.csv
4,Gansu,Mainland China,2020-01-22 17:00:00,NaN,NaN,NaN,36.0611,103.8343,01-22-2020.csv
...,...,...,...,...,...,...,...,...,...
213,Alaska,US,2020-03-10 02:33:04,0.0,0.0,0.0,61.3707,-152.4044,03-12-2020.csv
214,Idaho,US,2020-03-10 02:33:04,0.0,0.0,0.0,44.2405,-114.4788,03-12-2020.csv
215,Maine,US,2020-03-10 02:33:04,0.0,0.0,0.0,44.6939,-69.3819,03-12-2020.csv
216,West Virginia,US,2020-03-10 02:33:04,0.0,0.0,0.0,38.4912,-80.9545,03-12-2020.csv
